<a href="https://colab.research.google.com/github/yexf308/AppliedStochasticProcess/blob/main/MC_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Markov Chain Implentation from Scratch

In [1]:
import numpy as np

## homogeneous, discrete, finite state, first order Markov chain
## Implementation from scratch

In [30]:
class SimpleMarkovModel(object):
    def __init__(self, status_num=None):
        # initial probability vector
        self.pi = np.zeros(shape=(status_num))
        # transition probability matrix
        self.P = np.zeros(shape=(status_num, status_num))

    def fit(self, x):
        """
        Based on training data, calculate initial probability vector and transition probability matrix
        param x: x can be a single list or list of list. [s1, s2, ..., sn] or [[s11,s12,...,s1m],[s21,s22,...,s2n],...]
        The difference is in calculating initial probability vector. In single list, we can use all states to inference the initial
        prob vector. In list of list, we will use the initial states of sub-list to inference.
        return:
        """
        if type(x[0]) == list:
            for clist in x:
                self.pi[clist[0]] += 1
                for cindex in range(0, len(clist) - 1):
                    self.P[clist[cindex ], clist[cindex + 1]] += 1
        else:
            for index in range(0, len(x) - 1):
                self.pi[x[index]] += 1
                self.P[x[index ], x[index + 1]] += 1
        # normalization
        self.pi = self.pi / np.sum(self.pi)
        self.P = self.P / np.sum(self.P, axis=1)[:, np.newaxis]  


    def predict_log_joint_prob(self, status_list):
        """
        calculate the log of the joint probability
        param: status_list:
        :return:
        """
        log_prob = np.log(self.pi[status_list[0], 0])
        for index in range(0, len(status_list) - 1):
            log_prob += np.log(self.P[status_list[index], status_list[index + 1]])
        return log_prob  


    def predict_prob_distribution(self, time_steps=None, set_init_prob=None, set_prob_trans_matrix=None):
        """
        calculate the prob distribution after time_steps.
        Allow set_init_prob and set_prob_trans_matrix to set initial prob and prob_trans_matrix
        :param time_steps:
        :param set_init_prob:
        :param set_prob_trans_matrix:
        :return:
        """
        prob = self.pi if set_init_prob is None else set_init_prob
        trans_matrix = self.P if set_prob_trans_matrix is None else set_prob_trans_matrix
        for _ in range(0, time_steps):
            prob = np.matmul(prob, trans_matrix)
        return prob  

    def predict_next_step_prob_distribution(self, current_status=None):
        """
        predict next step probility distribution
        :param current_status:
        :return:
        """
        return self.P[[current_status], :]  

    def predict_next_step_status(self, current_status=None):
        """
        predict the most probable state in the next step
        :param current_status:
        :return:
        """
        return np.argmax(self.predict_next_step_prob_distribution(current_status))            




In [31]:
# weather prediction
# 0 - sunny, 1 - cloudy, 2 - rainy

train_data=[2,1,2,1,0,0,0,0,0,0,0,1,1,2,2,1,1,1,0,0,0,0,1,0,1,1,1,1,1,1]
smm=SimpleMarkovModel(status_num=3)
smm.fit(train_data)

In [32]:
smm.pi

array([0.4137931 , 0.44827586, 0.13793103])

In [25]:
P = np.random.rand(3,3)
b = np.array([1, 2,3])
print(P)
print(np.matmul(P, b))
print(np.matmul(b, P))



[[0.0171227  0.01821663 0.26378782]
 [0.35144182 0.02286129 0.0037491 ]
 [0.38484795 0.56941222 0.13140344]]
[0.84491942 0.40841171 1.9178827 ]
[1.87455019 1.77217586 0.66549634]


Let's check the probability distribution after 3, 5, 7 ,10, 20 days.

In [34]:
import pandas as pd
pd.DataFrame({"3nd day":smm.predict_prob_distribution(3).reshape(-1).tolist(),
              "5th day":smm.predict_prob_distribution(5).reshape(-1).tolist(),
              "7th day":smm.predict_prob_distribution(7).reshape(-1).tolist(),
              "10th day":smm.predict_prob_distribution(10).reshape(-1).tolist(),
              "20th day":smm.predict_prob_distribution(20).reshape(-1).tolist()})

,3nd day,5th day,7th day,10th day,20th day
0,0.426648,0.431260,0.432870,0.433556,0.433734
1,0.474763,0.471585,0.470475,0.470002,0.469880
2,0.098590,0.097155,0.096654,0.096441,0.096386


Let's try different initial conditions.


In [36]:
pd.DataFrame({"sunny":           smm.predict_prob_distribution(20,set_init_prob=np.asarray([1,0,0])).reshape(-1).tolist(),
              "cloudy":          smm.predict_prob_distribution(20,set_init_prob=np.asarray([0,1,0])).reshape(-1).tolist(),
              "rainy":           smm.predict_prob_distribution(20,set_init_prob=np.asarray([0,0,1])).reshape(-1).tolist(),
              "cloudy and rainy":smm.predict_prob_distribution(20,set_init_prob=np.asarray([0,0.5,0.5])).reshape(-1).tolist(),
              "Any":             smm.predict_prob_distribution(20,set_init_prob=np.asarray([0.05,0.2,0.75])).reshape(-1).tolist()})

,sunny,cloudy,rainy,cloudy and rainy,Any
0,0.433749,0.433726,0.433715,0.43372,0.433719
1,0.469870,0.469886,0.469893,0.46989,0.469891
2,0.096381,0.096388,0.096392,0.09639,0.096391
